# Hong Kong Reddit - Sentiment Analysis DashBoard (Part 3)

In [1]:
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import statistics as s
import pandas as pd
import re
import plotly.plotly as py
import plotly.graph_objs as go

c:\users\lawko698\documents\projects\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Load the data

The final data from part 1, where comments are attached to their respective posts will be used. 

In [2]:
with open('hongkong_complete.json','r') as f:
    complete = json.load(f)

In [3]:
# data format check
complete[1:2]

[{'author': 'restless_vagabond',
  'author_flair_css_class': None,
  'author_flair_text': None,
  'created_utc': 1276345654,
  'domain': 'self.HongKong',
  'full_link': 'https://www.reddit.com/r/HongKong/comments/ce8z5/anyone_up_for_the_reddit_meetup_next_saturday_196/',
  'id': 'ce8z5',
  'is_self': True,
  'num_comments': 4,
  'over_18': False,
  'permalink': '/r/HongKong/comments/ce8z5/anyone_up_for_the_reddit_meetup_next_saturday_196/',
  'score': 3,
  'selftext': 'so far there are 2 people who are thinking about meeting up.  If we get a few more we can decide on a central location.  ',
  'subreddit': 'HongKong',
  'subreddit_id': 't5_2rbn0',
  'thumbnail': 'self',
  'title': 'Anyone up for the Reddit Meetup next Saturday (19/6)?',
  'url': 'http://www.reddit.com/r/HongKong/comments/ce8z5/anyone_up_for_the_reddit_meetup_next_saturday_196/',
  'comments': [{'author': 'braedonsl',
    'author_flair_css_class': None,
    'author_flair_text': None,
    'body': "well i've got a couple f

We will implement the following json data structure to store information:

We will be using a sentiment analyzer called 'Vader' that is a parsimonious rule-based model for sentiment analysis of [social media text](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/download/8109/8122). Given how social media text is full of memes and emoticons, Vader's performance is cited to outperform individual human raters. 

Sentiment analyzer Vader doesn't require much text processing as it removes stop words and retains punctuations as part of its analysis. More examples can be found in this [blog](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html) 

In [4]:
# Initialize vader
vader = SentimentIntensityAnalyzer()

# Store each reddit post after sentiment analysis
sentiment_list = []

for submission in complete:
    if 'comments' in submission:
        temp_dict = {}

        temp_dict['full_link'] = submission['full_link']
        temp_dict['created_utc'] = submission['created_utc']
        temp_dict['url'] = submission['url']
        
        if submission['title'].strip() != '':
            result = re.sub(r"\n|\r|\t", "", submission['title'])
            temp_dict['title'] = result
        else:
            temp_dict['title'] = None
            
        if 'selftext' in submission:
            if submission['selftext'].strip() != '': #must not be empty else error when upload to database
                result = re.sub(r"\n|\r|\t", "", submission['selftext'])
                temp_dict['selftext'] = result
            else:
                temp_dict['selftext'] = None
        else:
            temp_dict['selftext'] = None

        temp_dict['comments'] = {}
        
        sentiment_compound_number = []
        sentiment_pos_number = []
        sentiment_neg_number = []
        sentiment_neu_number = []
        num_sentiment_type = [0,0,0]
        scores = []
        sentences = []
        
        for sentence in submission['comments']:
            result = re.sub(r"\n|\r|\t", "", sentence['body'])
          
            sentences.append(result)
            scores.append(sentence['score'])
            ss = vader.polarity_scores(result)
            for k in ss:
                if k == 'neg':
                    sentiment_neg_number.append(ss[k])
                elif k == 'neu':
                    sentiment_neu_number.append(ss[k])
                elif k == 'pos':
                    sentiment_pos_number.append(ss[k])
                elif k == 'compound':
                    sentiment_compound_number.append(ss[k])
                    if ss[k] > 0.2:
                        num_sentiment_type[0] += 1
                    elif ss[k] > -0.2 and ss[k] < 0.2:
                        num_sentiment_type[1] += 1
                    elif ss[k] < -0.2:
                        num_sentiment_type[2] += 1
                        
        temp_dict['comments']['sentences'] = sentences
        temp_dict['comments']['scores'] = scores
        temp_dict['comments']['sentiment_pos'] = sentiment_pos_number
        temp_dict['comments']['sentiment_neu'] = sentiment_neu_number
        temp_dict['comments']['sentiment_neg'] = sentiment_neg_number
        temp_dict['comments']['sentiment_compound'] = sentiment_compound_number
        temp_dict['comments']['num_sentiment_type'] = num_sentiment_type
      
        sentiment_list.append(temp_dict)

In [5]:
# check a random result
sentiment_list[13931]

{'full_link': 'https://www.reddit.com/r/HongKong/comments/6ikhy7/唔該/',
 'created_utc': 1498033395,
 'url': 'https://www.reddit.com/r/HongKong/comments/6ikhy7/唔該/',
 'title': '唔該',
 'selftext': 'Why is there three meanings for the exact same phrase: "唔該"? It could mean please, thanks and excuse meand why are there two iterations of thanks: 唔該 and 多謝Sincerely, a confused child',
 'comments': {'sentences': ['I tend to think of 唔該 as a courtesy phrase, something you say when people do you a favour.It means "excuse me" because it should be "唔該借借", which roughly translates to "please move aside". 多謝 Is generally reserved for gifting and wishes. It literally translates into "many thanks". You will never say "多謝" to someone holding the door open for you. Similarly, you will never say "唔該“ to someone giving you a gift. Not that I\'m a cantonese expert. Just giving my 2 cents. ',
   'This is Cantonese...With different tone and word combination i can even makes 唔該 have negative meaning唔該你啦 留心d聽下人

## Checkpoint: Output file to csv file

In the end, we found the following attributes useful for our analysis:
- compound: overall polarity score of sentiment (-1 to 1 where a score toward -1 is negative and score toward 1 is positive)
- num_positive: number of positive comments where compound number is greater than 0.2 
- num_neutral: number of neutral comments where compound number is between -0.2 to 0.2
- num_negative: number of negative comments where compound number is less than -0.2

In [6]:
with open('sentiment.csv','w', encoding="utf-8") as outfile:
    outfile.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('full_link','created_utc','title','url','selftext',
                                                                  'compound','num_positive','num_neutral','num_negative'))
    for topic in sentiment_list:
        outfile.write('\"{}\"\t{}\t\"{}\"\t\"{}\"\t{}\t{}\t{}\t{}\t{}\n'.format(topic['full_link'],
                                                            topic['created_utc'],topic['title'],
                                                            topic['url'],topic['selftext'],
                                                            s.mean(topic['comments']['sentiment_compound']),
                                                            topic['comments']['num_sentiment_type'][0],
                                                            topic['comments']['num_sentiment_type'][1],
                                                            topic['comments']['num_sentiment_type'][2]))

## Read csv file into Pandas DataFrame

In [7]:
df = pd.read_csv('sentiment.csv', encoding="utf-8", sep='\t' , index_col=False)

In [8]:
df.head()

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative
0,https://www.reddit.com/r/HongKong/comments/ce8...,1276345654,Anyone up for the Reddit Meetup next Saturday ...,http://www.reddit.com/r/HongKong/comments/ce8z...,so far there are 2 people who are thinking abo...,0.554533,3,0,0.0
1,https://www.reddit.com/r/HongKong/comments/dfx...,1284894399,Any redditors in Hong Kong?,http://www.reddit.com/r/HongKong/comments/dfx9...,I am doing an internship at the US consulate f...,0.327487,13,10,1.0
2,https://www.reddit.com/r/HongKong/comments/di5...,1285299510,48 hour film project... anyone doing this?,http://www.48hourfilm.com/hongkong/english.php,None,0.373017,5,1,0.0
3,https://www.reddit.com/r/HongKong/comments/dsm...,1287372820,"/r/hk, I was on YouTube when I was reminded of...",http://www.youtube.com/watch?v=5ZdEyK1rC8c,None,0.143600,1,1,1.0
4,https://www.reddit.com/r/HongKong/comments/dzz...,1288698370,Watch out: Predators on the Hong Kong MTR?,http://www.reddit.com/r/HongKong/comments/dzzu...,...cross posted from [/r/pics](http://www.redd...,0.472800,2,1,0.0


### Data Check for missing data

In [9]:
df[df['compound'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [10]:
df[df['selftext'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [11]:
df[df['title'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [12]:
df[df['full_link'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [13]:
df[df['url'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [14]:
df[df['num_positive'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


In [15]:
df[df['num_neutral'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative


It seems there are some inconsistencies from the wrangling. In three rows of the data, the columns starting from url has been shifted to the left. After examining the csv file, I couldn't find the reason for the shift(as other rows share similar structure). Fortunately, there is a simple solution to fix it.

In [16]:
df[df['num_negative'].isnull()]

,full_link,created_utc,title,url,selftext,compound,num_positive,num_neutral,num_negative
5883,https://www.reddit.com/r/HongKong/comments/2g2...,1410402551,Beijing official to HK democrats: “That you're...,None,-0.057977777777777774,8.0,8,11,NaN
12153,https://www.reddit.com/r/HongKong/comments/50f...,1472622247,Anson Chan speech: “The frustration on the par...,None,-0.18162666666666666,4.0,3,8,NaN
16958,https://www.reddit.com/r/HongKong/comments/8pz...,1528618523,"In an article, SCMP publishes a map in Simplif...",None,-0.206925,1.0,4,3,NaN


At the end of the selftext sentence, there are double quotes. By removing one of them it will rectify the shift.   

In [17]:
#Read the csv file and output changes into a new csv file
with open('sentiment.csv', 'r', encoding="utf-8") as input_file, open('sentiment_adj.csv', 'w', encoding="utf-8") as output_file:
    for i, line in enumerate(input_file):
        if i in (5884, 12155, 16960):
            list_of_words = line.split('\t')
            length_of_word = len(list_of_words[2])
            list_of_words[2] = '"' + list_of_words[2][1:length_of_word-3] + '"'
            output_file.write("\t".join(list_of_words))
        else:
            output_file.write(line)

## Read csv file into Pandas DataFrame (again)

In [18]:
df = pd.read_csv('sentiment_adj.csv', encoding="utf-8", sep='\t' , index_col=False)

In [19]:
#extract year from epoch timestamp
df['year'] = pd.to_datetime(df['created_utc'],unit='s').dt.year
#drop column
df.drop(['created_utc'], axis=1, inplace=True)

In [20]:
#rename column to title
df.rename(columns={"i": "title"}, inplace = True)

In [21]:
df.head()

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year
0,https://www.reddit.com/r/HongKong/comments/ce8...,Anyone up for the Reddit Meetup next Saturday ...,http://www.reddit.com/r/HongKong/comments/ce8z...,so far there are 2 people who are thinking abo...,0.554533,3,0,0,2010
1,https://www.reddit.com/r/HongKong/comments/dfx...,Any redditors in Hong Kong?,http://www.reddit.com/r/HongKong/comments/dfx9...,I am doing an internship at the US consulate f...,0.327487,13,10,1,2010
2,https://www.reddit.com/r/HongKong/comments/di5...,48 hour film project... anyone doing this?,http://www.48hourfilm.com/hongkong/english.php,None,0.373017,5,1,0,2010
3,https://www.reddit.com/r/HongKong/comments/dsm...,"/r/hk, I was on YouTube when I was reminded of...",http://www.youtube.com/watch?v=5ZdEyK1rC8c,None,0.143600,1,1,1,2010
4,https://www.reddit.com/r/HongKong/comments/dzz...,Watch out: Predators on the Hong Kong MTR?,http://www.reddit.com/r/HongKong/comments/dzzu...,...cross posted from [/r/pics](http://www.redd...,0.472800,2,1,0,2010


### Re-check Data

In [22]:
df[df['year'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [23]:
df[df['compound'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [24]:
df[df['selftext'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [25]:
df[df['title'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [26]:
df[df['full_link'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [27]:
df[df['url'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [28]:
df[df['num_positive'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


In [29]:
df[df['num_neutral'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


Problem Solved.

In [30]:
df[df['num_negative'].isnull()]

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year


## Visualize the overall Sentiment of HK reddit
Let's take a look at the overall trend in sentiment of Hong Kong reddit.

In [31]:
#group by year and sum all comments by sentiment
df_gp_year = df.groupby(['year']).sum()

In [32]:
trace1 = go.Bar(
    x=df_gp_year.index,
    y=df_gp_year['num_negative'],
    name='negative'
)
trace2 = go.Bar(
    x=df_gp_year.index,
    y=df_gp_year['num_neutral'],
    name='neutral'
)
trace3 = go.Bar(
    x=df_gp_year.index,
    y=df_gp_year['num_positive'],
    name='positive'
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~lawko698/0 or inside your plot.ly account where it is named 'grouped-bar'


## Focus on Media Posts

There are a range of subjects discussed on reddit from visiting Hong Kong to asking for advice about living in Hong Kong, but these are mundane topics. Lately, Hong Kong current events have become more political after the event of occupy central. Questions such as, what is the most widely discussed news article? How does the community feel about the outcome/event? 

Since we are interested in redditor's sentiment to current events in Hong Kong, we will filter posts that only include links to news article.

In [33]:
df_media = df[~df["url"].str.contains('reddit', na=False)].reset_index(drop=True)
df_media.head()

,full_link,title,url,selftext,compound,num_positive,num_neutral,num_negative,year
0,https://www.reddit.com/r/HongKong/comments/di5...,48 hour film project... anyone doing this?,http://www.48hourfilm.com/hongkong/english.php,None,0.373017,5,1,0,2010
1,https://www.reddit.com/r/HongKong/comments/dsm...,"/r/hk, I was on YouTube when I was reminded of...",http://www.youtube.com/watch?v=5ZdEyK1rC8c,None,0.143600,1,1,1,2010
2,https://www.reddit.com/r/HongKong/comments/e7e...,"Yesterday in Times Square. I'm sorry, I didn't...",http://imgur.com/emu6m.jpg,None,0.000000,0,1,0,2010
3,https://www.reddit.com/r/HongKong/comments/e80...,Again??,http://www.bbc.co.uk/news/world-asia-pacific-1...,None,-0.510600,0,0,1,2010
4,https://www.reddit.com/r/HongKong/comments/e9i...,"Free? If I was them, I would've charged $10 fo...",http://imgur.com/TZUVy.jpg,None,0.000000,0,5,0,2010


In [34]:
df_media.drop(['selftext'], axis = 1, inplace=True)

Currently, we will focus on Hong Kong News outlet. Futher work can be done to analyze overseas news articles.

Create new dataframe for analysis.

Note: Please ignore the warnings as the following processes do not have a problem with python's copy mechanism.

In [35]:
# filter df for scmp articles
df_focus_media = df_media[df_media["url"].str.contains('www.scmp.com', na=False)]
#create new media label
df_focus_media.loc[:,"media_name"] = 'scmp'

c:\users\lawko698\documents\projects\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\lawko698\documents\projects\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [36]:
# continue to append other media posts
for media in [('www.hongkongfp.com','hongkongfp'),('http://rthk.hk','rthk'),('www.ejinsight.com','ejinsight'),
              ('apple.nextmedia.com','hkdaily'),('thestandard.com.hk','thestandard'),
             ('hongwrong.com','hongwrong'),('timeout.com.hk','timeout'),('news.yahoo.com','yahoo')]:
    concat_media = df_media[df_media["url"].str.contains(media[0], na=False)]
    concat_media.loc[:,"media_name"] = media[1]
    df_focus_media = pd.concat([df_focus_media, concat_media])

c:\users\lawko698\documents\projects\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\users\lawko698\documents\projects\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [37]:
#create new ratio attribute of positive to negative comments
df_focus_media.loc[:,'pos_neg_ratio'] = (df_focus_media['num_positive'] + 1)/(df_focus_media['num_negative'] + 1)
#create new total comments attribute
df_focus_media.loc[:,'total_comments'] = (df_focus_media['num_positive'])+(df_focus_media['num_negative'])+(df_focus_media['num_neutral'])

In [38]:
#reset the index
df_focus_media.reset_index(drop=True, inplace=True)

In [39]:
#rearrange data for database upload
df_focus_media = df_focus_media.reindex(['media_name','title','full_link','url', 'compound', 'num_positive', 'num_neutral',
       'num_negative', 'year', 'pos_neg_ratio','total_comments'],axis=1)

In [40]:
# shift index by +1
df_focus_media.index += 1

In [41]:
# convert field to integer
df_focus_media['total_comments'] = df_focus_media['total_comments'].apply(lambda x: int(x))

In [42]:
#output df to csv file
df_focus_media.to_csv("reddit_media.csv", encoding="utf-8", header=False, sep='\t')